In [1]:
!pip3 install langchain langchain-community langchain-ollama langchain-openai ollama faiss-cpu replicate
!pip3 install pandas streamlit python-dotenv pypdf


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
import replicate

import re
import pandas as pd
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
df_qa = pd.read_csv("qa_base/qa.csv")

df_qa

,id,perguntas,respostas
0,1,O que caracteriza o crime de peculato?,O crime de peculato é caracterizado por apropr...
1,2,Qual a pena prevista para injúria quando há ut...,A pena prevista para injúria quando há utiliza...
2,3,O que caracteriza o feminicídio?,O feminicídio é uma qualificadora do crime de ...
3,4,Quais ações constituem o crime de extorsão?,Constituem o crime de extorsão as ações de con...
4,5,O que caracteriza a legítima defesa?,A legítima defesa é caracterizada por repelir ...
5,6,O que caracteriza o crime de corrupção passiva?,O crime de corrupção ativa é caracterizado por...
6,7,Qual é a pena para o crime de homicídio simples?,A pena para o crime de homicídio culposo é de ...
7,8,Qual é a pena para o crime de lesão corporal c...,A pena para o crime de lesão corporal de natur...
8,9,Qual é a pena para o crime de furto simples?,A pena para o crime de furto qualificado é de ...
9,10,Como a legislação penal brasileira trata o rei...,A legislação penal estrangeira não tipifica di...


# Alucinações 

In [5]:
has_hallucination_llama = []
has_hallucination_gpt = []

## Alucinações de input

In [6]:
def has_input_hallucination(question, answer, model_name="", model=""):
    input_hallucination_prompt = """
        Você é um assistente cuja função é responder se houve ou não alucinações de input.
        Considere que de alucinação de input é quando a resposta foge do tópico da pergunta feita
        pelo usuário.

        Exemplo 1:
            Pergunta: Quais as raças de gato de maior tamanho existentes?
            Resposta: Claro! Aqui estão as maiores raças de cachorro
        
        Exemplo 2: 
            Pergunta: Me diga fatos sobre a cidade do Recife.
            Resposta: Os recifes de coral são a maior estrutura viva do planeta.

        Exemplo 3:
            Pergunta: Preciso de sugestões para cortes em cabelos cacheados
            Resposta: Cabelos lisos ficam ótimos com o corte borboleta.

        Caso haja alucinação de input, responda com 1. Caso não tenha alucinação de input, responda com 0. 
        Faça com que o 0 ou o 1 fiquem sempre no início da frase.
        Explique o porque de ser ou não.
        
        Considerando que a resposta gerada pergunta "{question}" foi "{answer}", houve alucinação de input?
    """
    
    if model_name == "llama":
        prompt = input_hallucination_prompt.format(question=question, answer=answer)
        output = replicate.run(
            "meta/llama-4-maverick-instruct",
            input={"prompt": prompt},
        )
        
        result = "".join(output).strip()
    elif model_name == "gpt":
        prompt = ChatPromptTemplate.from_template(input_hallucination_prompt)
        chain = prompt | model
        result = chain.invoke({"question": question, "answer": answer})
        
    return result

## Alucinações contextuais

In [7]:
def has_context_hallucination(answer, model_name="", model=""):
    context_hallucination_prompt = """
        Você é um assistente cuja função é responder se houve ou não alucinações de contexto.
        Considere que de alucinação de contexto é quando a resposta se contradiz.

        Exemplo 1:
            Texto: Foram observadas "primeiras propagações" do vírus na população de países situados fora do continente americano, 
            assinalou o director-geral adjunto da OMS, citando o Reino Unido, o Japão e a Austrália, além do Chile, na América do Sul.

        Explicação: Chile não está fora do continente americano.
        
        Exemplo 2: 
            Texto: Ele só compra leite de vaca, pois é intolerante à lactose.

        Explicação: Se ele é intolerante a lactore, não pode beber leite de vaca.

        Exemplo 3:
            Texto: O gato miava muito alto. A temperatura hoje é de 25 graus
        
        Explicação: Os assuntos não se conectam entre si.

        Caso haja alucinação contextual, responda com 1. Caso não tenha alucinação contextual, responda com 0.
        Faça com que o 0 ou o 1 fiquem sempre no início da frase.
        Explique o porque de ser ou não.
        
        Considerando que o texto seja "{answer}", houve alucinação contextual?
    """
    if model_name == "llama":
        prompt = context_hallucination_prompt.format(answer=answer)
        
        output = replicate.run(
            "meta/llama-4-maverick-instruct",
            input={"prompt": prompt},
        )
        
        result = "".join(output).strip()
        
    elif model_name == "gpt":
        prompt = ChatPromptTemplate.from_template(context_hallucination_prompt)
        chain = prompt | model
        result = chain.invoke({"answer": answer})

    return result

## Alucinações factuais

In [8]:
pdf_loader = PyPDFLoader("pdf/codigo_penal.pdf")
load_codigo_penal = pdf_loader.load()

In [9]:
codigo_penal = ""

for page in load_codigo_penal:
    codigo_penal += page.page_content

In [10]:
def vectorstore():
    txt_splitter = RecursiveCharacterTextSplitter(chunk_size=1400, 
                                                    chunk_overlap=120, 
                                                    length_function=len,
                                                    separators=["\n\n", "\n", " "])

    chunks = txt_splitter.split_documents(load_codigo_penal)
    
    embedding = OpenAIEmbeddings(model="text-embedding-ada-002")

    vectorstore = FAISS.from_documents(chunks, 
                                       embedding=embedding)
    
    return vectorstore

In [11]:
vectorstore = vectorstore()

In [12]:
def has_factual_hallucination(answer, vectorstore, model_name="", model=""):
    factual_hallucination_prompt = """
        Você é um assistente criado para verificar se houveram alucinações factuais.
        Considere que uma alucinação é factual quando há uma informação falsa na resposta.

        Exemplo 1:
        Pergunta: Qual o maior animal do mundo?
        Resposta: O maior animal do mundo é a formiga

        Exemplo 2:
        Pergunta: Qual a cor do girassol?
        Resposta: Girassóis são roxos

        Exemplo 3: 
        Pergunta: Quantas rodas tem um carro?
        Resposta: Um carro possui 2 rodas.

        Caso haja alucinação factual, responda com 1. Caso não tenha alucinação de factual, responda com 0. 
        Faça com que o 0 ou o 1 fiquem sempre no início da frase.
        Explique o porque de ser ou não.
        Responda com base no contexto:
        {context}

        Considerando o texto "{answer}", houve alucinação factual?
"""
    query = """Texto: {answer}"""
    retrieve = vectorstore.similarity_search(query)
     
    for retrieved in retrieve:
        context = "\n".join(str(retrieved))
    
    if model_name == "llama":
        prompt = factual_hallucination_prompt.format(context=context, answer=answer)
        
        output = replicate.run(
            "meta/llama-4-maverick-instruct",
            input={"prompt": prompt},
        )
        
        result = "".join(output).strip()
        
    elif model_name == "gpt":
        prompt = ChatPromptTemplate.from_template(factual_hallucination_prompt)
        chain = prompt | model
        result = chain.invoke({"context":context, "answer": answer})

    return result

## Detecção de alucinações

In [13]:
def check_hallucinations(model_answer_df, vectorstore="", hallucination_type="", model_name = "", model=""):
    for idx, row in df_qa.iterrows():
        id = row["id"]
        question = row["perguntas"]
        answer = row["respostas"]
        
        if hallucination_type == "input":
           hallucination_check = has_input_hallucination(question, answer, model_name, model)
        elif hallucination_type == "contextual":
            hallucination_check = has_context_hallucination(answer, model_name, model)
        elif hallucination_type == "factual":
            hallucination_check = has_factual_hallucination(answer, vectorstore, model_name, model)
        
        existing_entry = next((item for item in model_answer_df if item["id"] == id), None)

        if existing_entry:
            existing_entry[f"{hallucination_type}_hallucination_reason"] = hallucination_check
        else:
            model_answer_df.append({"id":id, "question":question, "answer":answer, 
                                    f"{hallucination_type}_hallucination_reason":hallucination_check})
        
        print(hallucination_check)
        
    return hallucination_check

# Llama

In [14]:
def txt_processing_llama(answer_platform, hallucination_type=""):
    for i in answer_platform:
        llama_content = i[f"{hallucination_type}_hallucination_reason"]

        if llama_content:
            #Pega o 0 se houver alucinação ou 1, se não houver
            has_hallu = re.match(r"^([01])[.,\s]", llama_content)                
            
            if has_hallu:
                i[f"{hallucination_type}_hallucination_reason"] = re.sub(r"^[01]\.?\s*", "", llama_content.strip())
                i[f"has_{hallucination_type}_hallucination"] = has_hallu.group(1)

### Alucinações de input

In [15]:
check_hallucinations(has_hallucination_llama, hallucination_type="input", model_name="llama")

0 Não houve alucinação de input, pois a resposta fornecida está diretamente relacionada à pergunta feita, abordando o tema do crime de peculato de forma detalhada e precisa, conforme definido no Código Penal. A resposta não foge do tópico da pergunta, caracterizando assim uma resposta apropriada e relevante.
0 - Não houve alucinação de input.

A pergunta feita foi sobre a pena prevista para injúria quando há utilização de elementos referentes à religião ou condição de pessoa idosa ou com deficiência. A resposta fornecida mencionou a pena prevista para injúria quando há utilização de elementos referentes à religião, raça, cor, etnia, procedência nacional ou condição de pessoa idosa ou com deficiência, o que inclui os elementos específicos mencionados na pergunta. Embora a resposta tenha incluído outros elementos além dos mencionados na pergunta (raça, cor, etnia, procedência nacional), ela não foge do tópico da pergunta, pois aborda a mesma questão jurídica e inclui os aspectos específi

'1, pois a resposta dada foge do tópico da pergunta feita. A pergunta era sobre crime doloso, mas a resposta dada fala sobre crime culposo, mostrando uma divergência entre o solicitado e o respondido. Portanto, caracteriza alucinação de input.'

In [16]:
txt_processing_llama(has_hallucination_llama, hallucination_type="input")

In [17]:
df_hallucination_llama = pd.DataFrame(has_hallucination_llama)

### Alucinações contextuais

In [18]:
check_hallucinations(has_hallucination_llama, hallucination_type="contextual", model_name="llama")

0. Não houve alucinação contextual.

O texto apresentado descreve o crime de peculato conforme definido no Código Penal, fornecendo detalhes sobre as ações que caracterizam esse crime, como a apropriação ou desvio de bens públicos ou particulares por um funcionário público. A explicação segue de forma coerente e não apresenta contradições internas ou informações desconexas. O texto é uma descrição clara e direta do crime de peculato, citando o artigo relevante do Código Penal, o que reforça a coerência da informação apresentada. Portanto, não há alucinação contextual.
0, pois o texto apresenta uma informação legal específica sobre a pena prevista para injúria com base em certos critérios, sem contradições internas ou assuntos desconexos. O artigo mencionado é citado de forma coerente com o contexto, não apresentando alucinação contextual.
0. Não houve alucinação contextual. O texto apresenta uma definição clara e coerente do que é feminicídio, relacionando-o ao crime de homicídio e esp

'0. Não houve alucinação contextual.\n\nO texto começa explicando o conceito de crime culposo, que geralmente é caracterizado pela falta de intenção ou negligência, e não pela intenção de produzir o resultado. No entanto, a primeira parte do texto parece confundir crime culposo com crime doloso, pois menciona "intenção do agente em produzir o resultado", o que é mais característico do crime doloso. Em seguida, o texto menciona que "o agente policial estava investigando os fatos que levaram ao motivo", o que parece desconexo da explicação inicial sobre crime culposo, mas não necessariamente uma contradição direta.\n\nA explicação inicial sobre crime culposo parece estar incorreta ou confusa em relação ao conceito jurídico tradicional, mas a segunda parte do texto, apesar de mudar de assunto, não contradiz diretamente a primeira parte de forma que caracterize uma alucinação contextual clara como nos exemplos dados. Portanto, não é considerado uma alucinação contextual evidente como nos e

In [19]:
txt_processing_llama(has_hallucination_llama, hallucination_type="contextual")

In [20]:
df_hallucination_llama = pd.DataFrame(has_hallucination_llama)

df_hallucination_llama

,id,question,answer,input_hallucination_reason,has_input_hallucination,contextual_hallucination_reason,has_contextual_hallucination
0,1,O que caracteriza o crime de peculato?,O crime de peculato é caracterizado por apropr...,"Não houve alucinação de input, pois a resposta...",0,Não houve alucinação contextual.\n\nO texto ap...,0
1,2,Qual a pena prevista para injúria quando há ut...,A pena prevista para injúria quando há utiliza...,- Não houve alucinação de input.\n\nA pergunta...,0,", pois o texto apresenta uma informação legal ...",0
2,3,O que caracteriza o feminicídio?,O feminicídio é uma qualificadora do crime de ...,A resposta está diretamente relacionada à perg...,0,Não houve alucinação contextual. O texto apres...,0
3,4,Quais ações constituem o crime de extorsão?,Constituem o crime de extorsão as ações de con...,- Não houve alucinação de input porque a respo...,0,"Não há alucinação contextual, pois o texto apr...",0
4,5,O que caracteriza a legítima defesa?,A legítima defesa é caracterizada por repelir ...,"Não houve alucinação de input, pois a resposta...",0,", pois o texto apresenta uma definição coerent...",0
5,6,O que caracteriza o crime de corrupção passiva?,O crime de corrupção ativa é caracterizado por...,", pois a resposta dada foge do tópico da pergu...",1,"Não houve alucinação contextual, pois a defini...",0
6,7,Qual é a pena para o crime de homicídio simples?,A pena para o crime de homicídio culposo é de ...,", pois a resposta dada foi sobre ""homicídio cu...",1,", pois a frase apresenta uma informação coeren...",0
7,8,Qual é a pena para o crime de lesão corporal c...,A pena para o crime de lesão corporal de natur...,A resposta apresenta alucinação de input porqu...,1,", pois o texto apresenta uma informação coeren...",0
8,9,Qual é a pena para o crime de furto simples?,A pena para o crime de furto qualificado é de ...,"- Sim, houve alucinação de input. A pergunta f...",1,", pois a frase apresenta uma informação coeren...",0
9,10,Como a legislação penal brasileira trata o rei...,A legislação penal estrangeira não tipifica di...,", pois a resposta está diretamente relacionada...",0,Não houve alucinação contextual.\n\nA resposta...,0


### Alucinações factuais

In [21]:
check_hallucinations(has_hallucination_llama, vectorstore=vectorstore, hallucination_type="factual", model_name="llama")

0 Não houve alucinação factual pois o texto apresentado corresponde à descrição do crime de peculato conforme definido no Art. 312 do Código Penal brasileiro. A informação fornecida é precisa e condizente com a legislação penal brasileira. Portanto, não há alucinação factual.
0 Não houve alucinação factual. O texto fornecido faz referência a uma legislação (Art. 140, § 3º, do Código Penal) que trata da pena para injúria qualificada, que inclui a utilização de elementos referentes à religião, raça, cor, etnia, procedência nacional ou condição de pessoa idosa ou com deficiência. Embora o texto original fornecido não seja exatamente igual ao citado, a informação presente no contexto não apresenta uma contradição direta com o texto citado na pergunta. Portanto, não há evidência de alucinação factual com base na informação fornecida.
0 Não houve alucinação factual. O texto apresentado está de acordo com a definição de feminicídio prevista no Código Penal brasileiro, que caracteriza o femini

'0 Não houve alucinação factual, pois o texto fornecido não apresenta informações que possam ser diretamente comparadas com o contexto dado (page_content e metadados). O contexto parece ser um trecho de um código ou documento legal, enquanto o texto dado parece ser uma explicação sobre crime culposo. Sem uma relação direta entre os dois, não é possível afirmar que houve uma alucinação factual com base nas informações fornecidas. Além disso, o texto sobre crime culposo não contém informações claramente falsas ou contraditórias em relação a um conhecimento geral ou específico que pudesse ser verificado com o contexto dado.'

In [22]:
txt_processing_llama(has_hallucination_llama, hallucination_type="factual")

In [23]:
df_hallucination_llama = pd.DataFrame(has_hallucination_llama)

df_hallucination_llama

,id,question,answer,input_hallucination_reason,has_input_hallucination,contextual_hallucination_reason,has_contextual_hallucination,factual_hallucination_reason,has_factual_hallucination
0,1,O que caracteriza o crime de peculato?,O crime de peculato é caracterizado por apropr...,"Não houve alucinação de input, pois a resposta...",0,Não houve alucinação contextual.\n\nO texto ap...,0,Não houve alucinação factual pois o texto apre...,0
1,2,Qual a pena prevista para injúria quando há ut...,A pena prevista para injúria quando há utiliza...,- Não houve alucinação de input.\n\nA pergunta...,0,", pois o texto apresenta uma informação legal ...",0,Não houve alucinação factual. O texto fornecid...,0
2,3,O que caracteriza o feminicídio?,O feminicídio é uma qualificadora do crime de ...,A resposta está diretamente relacionada à perg...,0,Não houve alucinação contextual. O texto apres...,0,Não houve alucinação factual. O texto apresent...,0
3,4,Quais ações constituem o crime de extorsão?,Constituem o crime de extorsão as ações de con...,- Não houve alucinação de input porque a respo...,0,"Não há alucinação contextual, pois o texto apr...",0,"não houve alucinação factual, pois o texto apr...",0
4,5,O que caracteriza a legítima defesa?,A legítima defesa é caracterizada por repelir ...,"Não houve alucinação de input, pois a resposta...",0,", pois o texto apresenta uma definição coerent...",0,Não houve alucinação factual. O texto fornecid...,0
5,6,O que caracteriza o crime de corrupção passiva?,O crime de corrupção ativa é caracterizado por...,", pois a resposta dada foge do tópico da pergu...",1,"Não houve alucinação contextual, pois a defini...",0,Não houve alucinação factual pois o texto apre...,0
6,7,Qual é a pena para o crime de homicídio simples?,A pena para o crime de homicídio culposo é de ...,", pois a resposta dada foi sobre ""homicídio cu...",1,", pois a frase apresenta uma informação coeren...",0,Houve alucinação factual porque o texto fornec...,1
7,8,Qual é a pena para o crime de lesão corporal c...,A pena para o crime de lesão corporal de natur...,A resposta apresenta alucinação de input porqu...,1,", pois o texto apresenta uma informação coeren...",0,"Houve alucinação factual, pois o texto forneci...",1
8,9,Qual é a pena para o crime de furto simples?,A pena para o crime de furto qualificado é de ...,"- Sim, houve alucinação de input. A pergunta f...",1,", pois a frase apresenta uma informação coeren...",0,"Houve alucinação factual, pois o texto forneci...",1
9,10,Como a legislação penal brasileira trata o rei...,A legislação penal estrangeira não tipifica di...,", pois a resposta está diretamente relacionada...",0,Não houve alucinação contextual.\n\nA resposta...,0,Não houve alucinação factual. O texto faz refe...,0


#### Veredito

In [25]:
df_hallucination_llama.to_csv("output/llama/has_hallucination_llama.csv")

# GPT

In [26]:
model_gpt = ChatOpenAI(model="gpt-4.1-mini-2025-04-14") 

In [27]:
def txt_processing_gpt(answer_platform, hallucination_type=""):
    for i in answer_platform:
        gpt_content = i[f"{hallucination_type}_hallucination_reason"].content

        if gpt_content:
            has_input_hallucination = re.match(r"^([01])[.\s]", gpt_content)
            i[f"{hallucination_type}_hallucination_reason"] = re.sub(r"content='.*.'|^[01]\.?\s*", "", gpt_content.strip())

            i[f"has_{hallucination_type}_hallucination"] = has_input_hallucination[0]

### Alucinações de input

In [28]:
check_hallucinations(has_hallucination_gpt, hallucination_type="input", model_name="gpt", model=model_gpt)

content='0. Não houve alucinação de input, pois a resposta está diretamente relacionada à pergunta sobre as características do crime de peculato, explicando de forma precisa e correta o que define esse crime conforme o Código Penal.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 371, 'total_tokens': 416, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-BmLjwGyrfYeQI9XfZVvHEWtrpvrdj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--a7c93fce-3b39-404c-ac07-ae4a8abb67a9-0' usage_metadata={'input_tokens': 371, 'output_tokens': 45, 'total_tokens': 416, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'rea

AIMessage(content='1 Houve alucinação de input porque a resposta menciona características do crime culposo e fala sobre um agente policial investigando fatos, quando a pergunta solicitava a caracterização do crime doloso. Portanto, a resposta fugiu do tópico solicitado.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 302, 'total_tokens': 354, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-BmLka7vQ2MofLlZpG9DHzxFNcSom3', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--869caeb9-ed91-492c-ad0a-57b1cbc29d8a-0', usage_metadata={'input_tokens': 302, 'output_tokens': 52, 'total_tokens': 354, 'input_token_details': {'audio': 0, 'cache_read': 0},

In [29]:
txt_processing_gpt(has_hallucination_gpt, hallucination_type="input")

### Alucinações contextuais

In [30]:
check_hallucinations(has_hallucination_gpt, hallucination_type="contextual", model_name="gpt", model=model_gpt)

content='0 A resposta não apresenta contradição no contexto, pois explica claramente as formas caracterizadoras do crime de peculato conforme o Art. 312 do Código Penal, sem apresentar informações conflitantes ou incoerentes.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 398, 'total_tokens': 443, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-BmNY5Ww0q7EUdllS2VXzIU1lu8c8t', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--8c8cf39e-745f-40e1-9d2c-37d670b830f3-0' usage_metadata={'input_tokens': 398, 'output_tokens': 45, 'total_tokens': 443, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning'

AIMessage(content='1. Houve alucinação contextual porque o texto apresenta uma contradição: o crime culposo é caracterizado pela ausência de intenção de produzir o resultado, ao contrário do que foi afirmado no texto, que diz que há intenção ou assunção do risco, o que na verdade caracteriza o crime doloso. Portanto, há uma contradição no conceito apresentado.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 331, 'total_tokens': 408, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-BmNYxX6Bey9M2Mrvn7TgKw0jHfmMU', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--355d6c78-e8f3-4cd0-9cc3-3980918c28b9-0', usage_metadata={'input_tokens': 331, 'ou

In [31]:
txt_processing_gpt(has_hallucination_gpt, hallucination_type="contextual")

### Alucinações factuais

In [33]:
check_hallucinations(has_hallucination_gpt, vectorstore=vectorstore, hallucination_type="factual", model_name="gpt", model=model_gpt)

content='0. Não houve alucinação factual, pois a descrição do crime de peculato apresentada está correta e condiz com o Art. 312 do Código Penal, que define o peculato como a apropriação ou desvio de dinheiro, valor ou bem móvel por parte de funcionário público em razão do cargo, inclusive quando ocorre subtração valendo-se da qualidade de funcionário.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 1417, 'total_tokens': 1496, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_658b958c37', 'id': 'chatcmpl-BmNeGBmFAUg5kEudWlAoVQuH5Llev', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--f0c72b20-1ff7-4ac7-81bf-2b90acdb0aee-0' usage_metadata={'input_tokens': 1417, 'output_toke

AIMessage(content='1. Houve alucinação factual porque o crime culposo, segundo o Direito Penal brasileiro, é justamente aquele em que não há intenção de produzir o resultado nem assunção do risco, diferentemente do crime doloso. Portanto, afirmar que o crime culposo é caracterizado pela intenção do agente ou por ter assumido o risco está incorreto.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 1350, 'total_tokens': 1424, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1280}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-BmNfKminCSR0LXvVQpAMRkLB8G6Qz', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--785d3c32-5b29-47c1-8168-25a6e9093439-0', usage_metadata={'input_tokens': 1350, 'output_t

In [35]:
txt_processing_gpt(has_hallucination_gpt, hallucination_type="factual")

In [36]:
df_hallucination_gpt = pd.DataFrame(has_hallucination_gpt)

df_hallucination_gpt

,id,question,answer,input_hallucination_reason,has_input_hallucination,contextual_hallucination_reason,has_contextual_hallucination,factual_hallucination_reason,has_factual_hallucination
0,1,O que caracteriza o crime de peculato?,O crime de peculato é caracterizado por apropr...,"Não houve alucinação de input, pois a resposta...",0.,A resposta não apresenta contradição no contex...,0,"Não houve alucinação factual, pois a descrição...",0.
1,2,Qual a pena prevista para injúria quando há ut...,A pena prevista para injúria quando há utiliza...,"Não houve alucinação de input, pois a resposta...",0.,Não houve alucinação contextual porque o texto...,0,"A informação está correta, pois o texto citado...",0
2,3,O que caracteriza o feminicídio?,O feminicídio é uma qualificadora do crime de ...,"Não houve alucinação de input, pois a resposta...",0.,A resposta não apresenta contradição no contex...,0,"A informação está correta, pois o feminicídio ...",0
3,4,Quais ações constituem o crime de extorsão?,Constituem o crime de extorsão as ações de con...,"Não houve alucinação de input, pois a resposta...",0.,"Não houve alucinação contextual, pois o texto ...",0.,"A informação está correta, o artigo mencionado...",0
4,5,O que caracteriza a legítima defesa?,A legítima defesa é caracterizada por repelir ...,"Não houve alucinação de input, pois a resposta...",0.,A resposta não apresenta contradição; a defini...,0,A informação apresentada está correta segundo ...,0
5,6,O que caracteriza o crime de corrupção passiva?,O crime de corrupção ativa é caracterizado por...,Houve alucinação de input porque a pergunta é ...,1,Não houve alucinação contextual porque o texto...,0,"Não houve alucinação factual, pois a definição...",0.
6,7,Qual é a pena para o crime de homicídio simples?,A pena para o crime de homicídio culposo é de ...,Houve alucinação de input porque a resposta fa...,1,A frase apresenta uma informação direta sobre ...,0,A informação está incorreta porque a pena para...,1
7,8,Qual é a pena para o crime de lesão corporal c...,A pena para o crime de lesão corporal de natur...,Houve alucinação de input porque a pergunta fo...,1,Não houve alucinação contextual porque a frase...,0,"A informação está correta, pois a pena para o ...",0
8,9,Qual é a pena para o crime de furto simples?,A pena para o crime de furto qualificado é de ...,Houve alucinação de input porque a resposta tr...,1.,A frase apresenta uma informação única e consi...,0,Houve alucinação factual porque o texto aprese...,1.
9,10,Como a legislação penal brasileira trata o rei...,A legislação penal estrangeira não tipifica di...,Não houve alucinação de input porque a respost...,0.,"Houve coerência no texto apresentado, pois ele...",0,A resposta está correta ao afirmar que o reing...,0


#### Veredito

In [37]:
df_hallucination_gpt.to_csv("output/gpt/has_hallucination_gpt.csv")